#  Langfuse Evaluation을 사용한 RAG 답변 평가

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

import uuid

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추적 활성화, false: langsmith 추적 비활성화)
print("langsmith 추적 여부: ", os.getenv('LANGSMITH_TRACING'))

langsmith 추적 여부:  true


`(3) Test Data`

In [4]:
# Test 데이터셋에 대한 QA 생성 결과를 리뷰한 후 다시 로드
import pandas as pd
df_qa_test = pd.read_excel("data/testset.xlsx")

print(f"테스트셋: {df_qa_test.shape[0]}개 문서")
df_qa_test.head(2)

테스트셋: 49개 문서


,user_input,reference_contexts,reference,synthesizer_name
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","['Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(...",single_hop_specifc_query_synthesizer
1,Forbes Global 2000에서 테슬라 순위 뭐야?,['Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (...,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.,single_hop_specifc_query_synthesizer


---

## **검색 도구 정의** 

### 1) **벡터스토어** 로드

- **Chroma DB** 설정에서 모델, 컬렉션명, 저장 경로 지정

In [5]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

In [6]:
# 벡터저장소 검색기 생성
chroma_k = chroma_db.as_retriever(
    search_kwargs={'k': 4},
)

# 벡터저장소 검색기를 사용하여 검색
query = "테슬라의 회장은 누구인가요?"

retrieved_docs = chroma_k.invoke(query)

# 검색 결과 출력
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("-"*200)
    print()

### 2) **BM25 검색기** 준비

- **BM25 검색기** 구현으로 문서 유사도 기반 검색 가능

- **한국어 텍스트 처리**를 위한 **Kiwi 토크나이저** 설정

- 참고: https://github.com/bab2min/kiwipiepy

In [7]:
# korean_docs 파일을 로드 (jsonlines 파일)
def load_jsonlines(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        docs = [json.loads(line) for line in f]
    return docs

korean_docs = load_jsonlines('data/korean_docs_final.jsonl')
print(f"로드된 문서: {len(korean_docs)}개")
pprint(korean_docs[0])

로드된 문서: 39개
('{"id":null,"metadata":{"source":"data/테슬라_KR.md","company":"테슬라","language":"ko"},"page_content":"<Document>\\nTesla, '
 'Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사는 전기 자동차(BEV), 고정형 배터리 에너지 저장 장치, 태양 '
 '전지판, 태양광 지붕널 및 관련 제품/서비스를 설계, 제조 및 판매합니다. 2003년 7월 Martin Eberhard와 Marc '
 'Tarpenning이 Tesla Motors로 설립했으며, Nikola Tesla를 기리기 위해 명명되었습니다. Elon Musk는 '
 '2004년 Tesla의 초기 자금 조달을 주도하여 2008년에 회장 겸 CEO가 '
 "되었습니다.\\n</Document>\\n<Source>이 문서는 미국 전기차 회사인 '테슬라'에 대한 "
 '문서입니다.</Source>","type":"Document"}')


In [8]:
from langchain.schema import Document  # Document 클래스 임포트

# 문자열 리스트를 Document 객체로 변환
if isinstance(korean_docs[0], str):  # 첫 번째 항목이 문자열인지 확인
    documents = [
        Document(
            page_content=json.loads(data)['page_content'],  # 문자열을 파이썬 객체로 변환
            metadata=json.loads(data)['metadata']
        ) 
        for i, data in enumerate(korean_docs)
    ]
else:
    documents = korean_docs

print(f"변환된 문서: {len(documents)}개")
pprint(documents[0])

변환된 문서: 39개
Document(metadata={'source': 'data/테슬라_KR.md', 'company': '테슬라', 'language': 'ko'}, page_content="<Document>\nTesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사는 전기 자동차(BEV), 고정형 배터리 에너지 저장 장치, 태양 전지판, 태양광 지붕널 및 관련 제품/서비스를 설계, 제조 및 판매합니다. 2003년 7월 Martin Eberhard와 Marc Tarpenning이 Tesla Motors로 설립했으며, Nikola Tesla를 기리기 위해 명명되었습니다. Elon Musk는 2004년 Tesla의 초기 자금 조달을 주도하여 2008년에 회장 겸 CEO가 되었습니다.\n</Document>\n<Source>이 문서는 미국 전기차 회사인 '테슬라'에 대한 문서입니다.</Source>")


In [9]:
# BM25 검색기를 사용하기 위한 준비
from krag.tokenizers import KiwiTokenizer
from krag.retrievers import KiWiBM25RetrieverWithScore

kiwi_tokenizer = KiwiTokenizer(
    model_type='knlm',    # Kiwi 언어 모델 타입
    typos='basic'         # 기본 오타교정
    )

bm25_db = KiWiBM25RetrieverWithScore(
        documents=documents, 
        kiwi_tokenizer=kiwi_tokenizer, 
        k=4, 
    )

In [10]:
# BM25 검색기를 사용하여 문서 검색
query = "테슬라의 회장은 누구인가요?"
retrieved_docs = bm25_db.invoke(query)

# 검색 결과 출력 
for doc in retrieved_docs:
    print(f"BM25 점수: {doc.metadata["bm25_score"]:.2f}")    
    print(f"\n{doc.page_content}\n[출처: {doc.metadata['source']}]")
    print("-"*200)

BM25 점수: 5.73

<Document>
Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사는 전기 자동차(BEV), 고정형 배터리 에너지 저장 장치, 태양 전지판, 태양광 지붕널 및 관련 제품/서비스를 설계, 제조 및 판매합니다. 2003년 7월 Martin Eberhard와 Marc Tarpenning이 Tesla Motors로 설립했으며, Nikola Tesla를 기리기 위해 명명되었습니다. Elon Musk는 2004년 Tesla의 초기 자금 조달을 주도하여 2008년에 회장 겸 CEO가 되었습니다.
</Document>
<Source>이 문서는 미국 전기차 회사인 '테슬라'에 대한 문서입니다.</Source>
[출처: data/테슬라_KR.md]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BM25 점수: 5.58

<Document>
Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, Musk의 논란의 여지가 있는 발언과 관련된 소송, 정부 조사 및 비판에 직면했습니다.

## 역사

### 창립 (2003–2004)

Tesla Motors, Inc.는 2003년 7월 1일에 Martin Eberhard와 Marc Tarpenning에 의해 설립되었으며, 각각 CEO와 CFO를 역임했습니다. Ian Wright는 얼마 지나지 않아 합류했습니다. 2004년 2월, Elon Musk는 750만 달러의 시리즈 A 자금 조달을 주도하여 회장 겸 최대 주주가 되었습니다. J. B. Straubel은 2004년 5월 CTO로 합류했습니다. 다섯 명 모두 공동 설립자로 인정받고 있습니다.



### 3) **Emsemble Hybrid Search** 준비

- **BM25**, **벡터 검색** 결과를 **rank-fusion** 알고리즘으로 통합 (**EnsembleRetriever**)

- 각 검색기의 **순위 점수**를 고려한 최종 순위 결정

- **중복 문서** 제거와 **재순위화** 자동 수행

- 두 검색 방식의 **장점을 결합**해 검색 품질 향상

In [11]:
from langchain.retrievers import EnsembleRetriever

# 검색기 초기화 
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_db, chroma_k],
    weights=[0.5, 0.5],
)

In [12]:
query = "테슬라의 회장은 누구인가요?"
retrieved_docs = hybrid_retriever.invoke(query)

# 검색 결과 출력 
for doc in retrieved_docs:
    print(f"\n{doc.page_content}\n[출처: {doc.metadata['source']}]")
    print("-"*200)


<Document>
Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사는 전기 자동차(BEV), 고정형 배터리 에너지 저장 장치, 태양 전지판, 태양광 지붕널 및 관련 제품/서비스를 설계, 제조 및 판매합니다. 2003년 7월 Martin Eberhard와 Marc Tarpenning이 Tesla Motors로 설립했으며, Nikola Tesla를 기리기 위해 명명되었습니다. Elon Musk는 2004년 Tesla의 초기 자금 조달을 주도하여 2008년에 회장 겸 CEO가 되었습니다.
</Document>
<Source>이 문서는 미국 전기차 회사인 '테슬라'에 대한 문서입니다.</Source>
[출처: data/테슬라_KR.md]
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

<Document>
Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, Musk의 논란의 여지가 있는 발언과 관련된 소송, 정부 조사 및 비판에 직면했습니다.

## 역사

### 창립 (2003–2004)

Tesla Motors, Inc.는 2003년 7월 1일에 Martin Eberhard와 Marc Tarpenning에 의해 설립되었으며, 각각 CEO와 CFO를 역임했습니다. Ian Wright는 얼마 지나지 않아 합류했습니다. 2004년 2월, Elon Musk는 750만 달러의 시리즈 A 자금 조달을 주도하여 회장 겸 최대 주주가 되었습니다. J. B. Straubel은 2004년 5월 CTO로 합류했습니다. 다섯 명 모두 공동 설립자로 인정받고 있습니다.

### Roadster (2005–2009)
</D

## **RAG Chain** 정의

- OpenAI gpt-4.1-mini 모델 활용

In [13]:
# 각 쿼리에 대한 검색 결과를 한꺼번에 Context로 전달해서 답변을 생성
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def create_rag_chain(retriever, llm):

    template = """Answer the following question based on this context. If the context is not relevant to the question, just answer with '답변에 필요한 근거를 찾지 못했습니다.'

    [Context]
    {context}

    [Question]
    {question}

    [Answer]
    """

    prompt = ChatPromptTemplate.from_template(template)

    def format_docs(docs):
        return "\n\n".join([f"{doc.page_content}" for doc in docs])

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()} 
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

In [14]:
# RAG 체인 생성 및 테스트
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.5)

openai_rag_chain = create_rag_chain(hybrid_retriever, llm)

question = "테슬라의 회장은 누구인가요?"
answer = openai_rag_chain.invoke(question)

print(f"쿼리: {question}")
print(f"답변: {answer}") 

쿼리: 테슬라의 회장은 누구인가요?
답변: Elon Musk가 테슬라의 회장입니다.


---

## **Langfuse를 활용한 평가**

1. **환경 설정**: Langfuse 클라이언트 초기화 및 인증
2. **데이터셋 업로드**: `create_dataset()` 및 `create_dataset_item()` 사용
3. **모니터링 RAG 체인**: `CallbackHandler`로 자동 추적 설정
4. **평가 실행**: 데이터셋 기반 체계적 평가

**주요 장점:**
- 🔄 자동화된 추적 및 로깅
- 📊 시각적 대시보드 제공
- 🔍 다양한 평가 지표 지원
- 🚀 확장 가능한 평가 파이프라인

---
### 1) **Langfuse 환경 설정** 

In [15]:
from langfuse.langchain import CallbackHandler

# Langfuse 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

In [16]:
from langfuse import get_client

# Langfuse 클라이언트 초기화
langfuse_client = get_client()

# 인증 확인
print("Langfuse 인증 상태:", langfuse_client.auth_check())

Langfuse 인증 상태: True


---
### 2) **평가용 데이터셋 업로드** 

In [17]:
df_qa_test.head(2)

,user_input,reference_contexts,reference,synthesizer_name
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","['Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(...",single_hop_specifc_query_synthesizer
1,Forbes Global 2000에서 테슬라 순위 뭐야?,['Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (...,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.,single_hop_specifc_query_synthesizer


In [18]:
# Langfuse에서 데이터셋 생성
name = "RAG_Evaluation_Dataset_Test"
dataset = langfuse_client.create_dataset(name="RAG_Evaluation_Dataset_Test")

print(f"생성된 데이터셋: {dataset.name}")

생성된 데이터셋: RAG_Evaluation_Dataset_Test


In [19]:
# 평가용 데이터셋을 변환 
data=[
    {
        "user_input": row["user_input"],
        "reference": row["reference"],
        "reference_contexts": row["reference_contexts"],
    } for _, row in df_qa_test.iterrows()
]

print(f"평가용 데이터셋 아이템 수: {len(data)}개")

평가용 데이터셋 아이템 수: 49개


In [20]:
# 데이터셋 아이템 추가
for item in data:
    langfuse_client.create_dataset_item(
        dataset_name=name,
        input=item.get("user_input", ""),
        expected_output=item.get("reference", ""),
        metadata={
            "reference_contexts": item.get("reference_contexts", ""),            }
    )


# langfuse에 플러시 (저장)
langfuse_client.flush()

In [21]:
# 추가된 데이터셋 확인 (가져오기)
dataset = langfuse_client.get_dataset(name="RAG_Evaluation_Dataset_Test")
print(f"생성된 데이터셋: {dataset.name}")
print(f"데이터셋 아이템 수: {len(dataset.items)}개")

생성된 데이터셋: RAG_Evaluation_Dataset_Test
데이터셋 아이템 수: 49개


In [22]:
# 평가용 데이터셋 아이템 출력
for item in dataset.items[:5]:  # 처음 5개 아이템만 출력
    print(f"입력: {item.input}")
    print(f"기대 출력: {item.expected_output}")
    print(f"메타데이터: {item.metadata}")
    print("-"*200)

입력: Elon Musk의 560억 달러 급여 패키지가 델라웨어 법원에서 거부된 이유와 그의 Tesla에서의 초기 전략적 역할은 무엇인가요?
기대 출력: Elon Musk의 560억 달러 급여 패키지는 2024년 12월 델라웨어 법원에서 부적절한 이사회 승인으로 인해 거부되었습니다. Tesla에서의 초기 전략적 역할로는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략에 초점을 맞추는 것이었습니다. 그는 Valor Equity Partners와 같은 투자자들로부터 자금을 조달하며 Tesla의 성장을 이끌었습니다.
메타데이터: {'reference_contexts': '[\'<1-hop>\\n\\n2024년 12월, 델라웨어 법원은 부적절한 이사회 승인을 이유로 Elon Musk의 560억 달러 급여 패키지를 거부했습니다.\\n\\n## 자동차 제품 및 서비스\\n\\n2024년 11월 현재 Tesla는 Model S, Model X, Model 3, Model Y, Semi 및 Cybertruck의 6가지 차량 모델을 제공합니다. 다음은 Tesla 모델 목록입니다.\', "<2-hop>\\n\\nElon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략에 초점을 맞춰 적극적인 역할을 수행했습니다. 후속 자금 조달에는 Valor Equity Partners (2006)와 Sergey Brin, Larry Page, Jeff Skoll과 같은 기업가의 투자가 포함되었습니다.\\n\\n2007년 8월, Eberhard는 CEO에서 물러나라는 요청을 받았고, Tarpenning은 2008년 1월에 이어졌습니다. Michael Marks는 Ze\'ev Drori가 인수하기 전에 임시 CEO를 역임했으며, Musk는 2008년 10월에 인수했습니다. Eberhard는 2009년 6월 Musk를 상대로 소송을 제기했지만 나중에 기각되었습니다."]'}
-------------------------------------------------

---
### 3) **데이터셋 기반 평가 실행** 

In [23]:
from langchain.evaluation import load_evaluator
from korouge_score import rouge_scorer
from krag.tokenizers import KiwiTokenizer

# Kiwi 토크나이저 사용하여 토큰화하는 클래스 정의 
class CustomKiwiTokenizer(KiwiTokenizer):
    def tokenize(self, text):
        return [t.form for t in super().tokenize(text)]

# 토크나이저 생성
kiwi_tokenizer = CustomKiwiTokenizer(model_type='knlm', typos='basic')

# ROUGE 스코어 계산
scorer = rouge_scorer.RougeScorer(
    ["rouge1", "rouge2", "rougeL"], 
    tokenizer=kiwi_tokenizer      # tokenize 메소드를 갖는 토크나이저 사용
)

# 평가자 로드 (간결성 평가)
conciseness_evaluator = load_evaluator(
    evaluator="labeled_criteria", 
    criteria="conciseness",
    llm=llm
)

In [24]:
from dataclasses import dataclass
from typing import Any, Dict, List, Optional
from langfuse.langchain import CallbackHandler
from langfuse import get_client

@dataclass
class EvaluationResult:
    """평가 결과를 담는 데이터 클래스"""
    item_id: str
    input: Any
    output: str
    expected_output: str
    scores: Dict[str, float]
    details: Dict[str, Any]
    trace_id: Optional[str] = None
    error: Optional[str] = None

def run_dataset_evaluation(dataset_name: str, rag_chain, run_name: str) -> List[EvaluationResult]:
    """데이터셋 전체에 대한 평가 실행"""

    # 데이터셋 가져오기
    langfuse_client = get_client()
    dataset = langfuse_client.get_dataset(name=dataset_name)
    if not dataset:
        raise ValueError(f"데이터셋 '{dataset_name}'이(가) 존재하지 않습니다.")
    
    print(f"📊 RAG 평가 시작: {dataset_name} ({len(dataset.items)}개 항목)")
    
    results = []
    successful = 0
    failed = 0
    
    for idx, item in enumerate(dataset.items, 1):
        try:
            print(f"\n🔄 아이템 {idx}/{len(dataset.items)} 처리 중...")
            
            # Langfuse 트레이싱 설정
            with item.run(run_name=run_name) as root_span:

                # RAG 체인 실행
                output = rag_chain.invoke(
                    item.input,
                    config={"callbacks": [CallbackHandler()]}  # Langfuse 콜백 핸들러 추가
                    )
                
                # 평가 수행
                scores, details = {}, {}

                # 1. ROUGE 점수 평가
                try:
                    rouge_results = scorer.score(
                        str(item.expected_output), 
                        str(output)
                    )
                    rouge_scores = {
                        "rouge1": rouge_results['rouge1'].fmeasure,
                        "rouge2": rouge_results['rouge2'].fmeasure,
                        "rougeL": rouge_results['rougeL'].fmeasure
                    }
                    scores["rouge"] = sum(rouge_scores.values()) / len(rouge_scores)
                    details["rouge"] = rouge_scores
                except Exception as e:
                    scores["rouge"] = 0.0
                    details["rouge"] = {"error": str(e)}

                # 2. 간결성 평가
                try:
                    conciseness_result = conciseness_evaluator.evaluate_strings(
                        input=str(item.input),
                        prediction=str(output),
                        reference=str(item.expected_output)
                    )
                    scores["conciseness"] = float(conciseness_result.get('score', 0))
                    details["conciseness"] = {
                        "reasoning": conciseness_result.get('reasoning', ''),
                        "score": conciseness_result.get('score', 0)
                    }
                except Exception as e:
                    scores["conciseness"] = 0.0
                    details["conciseness"] = {"error": str(e)}

                # 전체 점수 계산 및 기록
                overall_score = sum(scores.values()) / len(scores)
                root_span.score(name="overall", value=overall_score)
                
                # 각 평가 점수 기록
                for score_name, score_value in scores.items():
                    root_span.score(name=score_name, value=score_value)
                
                # 결과 저장
                result = EvaluationResult(
                    item_id=item.id,
                    input=item.input,
                    output=str(output),
                    expected_output=str(item.expected_output) if item.expected_output else "",
                    scores=scores,
                    details=details,
                    trace_id=getattr(root_span, 'trace_id', None)
                )
                results.append(result)
                successful += 1
                
                print(f"   ✅ 완료 (종합 점수: {overall_score:.2f})")
                print(f"   🔍 세부 정보: {details}")
                
        except Exception as e:
            failed += 1
            print(f"   ❌ 실패: {str(e)}")
            
            # 실패해도 결과에 기록
            results.append(EvaluationResult(
                item_id=item.id,
                input=item.input,
                output="",
                expected_output=str(item.expected_output) if item.expected_output else "",
                scores={},
                details={},
                error=str(e)
            ))
    
    # 결과 요약
    print(f"\n📋 평가 완료: 성공 {successful}개, 실패 {failed}개")
    
    return results


In [35]:
# 평가 실행
results = run_dataset_evaluation(
    dataset_name="RAG_Evaluation_Dataset_Test",  # 평가할 데이터셋 이름
    rag_chain=openai_rag_chain,  # 실제 RAG 체인
    run_name="simple_evaluation_v1" # 평가 실행 이름
)

📊 RAG 평가 시작: RAG_Evaluation_Dataset_Test (49개 항목)

🔄 아이템 1/49 처리 중...
   ✅ 완료 (종합 점수: 0.77)
   🔍 세부 정보: {'rouge': {'rouge1': 0.6871794871794872, 'rouge2': 0.455958549222798, 'rougeL': 0.5025641025641026}, 'conciseness': {'reasoning': 'Step-by-step reasoning about conciseness:\n\n1. The submission addresses both parts of the question: the reason for the rejection of Elon Musk\'s $56 billion pay package and his early strategic role at Tesla.\n\n2. The submission states the pay package was rejected due to "부적절한 이사회 승인을 이유로" (inappropriate board approval) and gives a specific date (December 2024), which aligns with the reference.\n\n3. Regarding Musk\'s early strategic role, the submission mentions:\n   - Leading early funding in 2004\n   - Focusing on starting with premium sports cars and expanding to mainstream vehicles\n   - Becoming chairman and CEO in 2008\n   - Contributing significantly to Tesla\'s growth and product development\n\n4. The reference is more concise, stating the rejec

---
### **[실습]**  

- Langfuse에 설정한 테스트 데이터에 대한 RAG를 수행할 수 있는 체인을 정의합니다. 
- LangChain의 `load_evaluator`를 활용하여 평가자를 로드하고, LLM-as-Judge 방식으로 생성된 답변의 품질을 평가합니다. 
- 각 답변의 평가 결과를 Langfuse에 기록합니다.
- Langfuse 대시보드에서 평가 결과를 시각적으로 분석합니다. 

In [34]:
# 1. 먼저 존재하는 데이터셋 목록을 확인
langfuse_client = get_client()
print("기존 데이터셋 목록:")
try:
    # 데이터셋 목록 확인 (Langfuse API가 지원하는 경우)
    pass
except:
    print("데이터셋 목록을 가져올 수 없습니다.")

# 2. 데이터셋이 없는 경우 다시 생성
try:
    # 데이터셋 생성
    dataset_name = "RAGEvaluationDatasetTest"
    dataset = langfuse_client.create_dataset(name=dataset_name)
    
    # 데이터 추가
    data = [
        {
            "user_input": row["user_input"],
            "reference": row["reference"],
            "reference_contexts": row["reference_contexts"],
        }
        for _, row in df_qa_test.iterrows()
    ]
    
    # 데이터셋 아이템 생성
    for item in data:
        langfuse_client.create_dataset_item(
            dataset_name=dataset_name,
            input=item.get("user_input", ""),
            expected_output=item.get("reference", ""),
            metadata={
                "reference_contexts": item.get("reference_contexts", "")
            },
        )
    
    # Langfuse에 반영
    langfuse_client.flush()
    
    print(f"데이터셋 '{dataset_name}' 생성 완료")
    
except Exception as e:
    print(f"데이터셋 생성 오류: {e}")

# 3. 이제 실험 실행
try:
    results = run_dataset_evaluation(
        dataset_name="RAGEvaluationDatasetTest",
        rag_chain=openai_rag_chain,
        run_name="simple_evaluation_v1"
    )
    print(f"실험 완료: {len(results)}개 결과")
except Exception as e:
    print(f"실험 실행 오류: {e}")


기존 데이터셋 목록:
데이터셋 'RAGEvaluationDatasetTest' 생성 완료
📊 RAG 평가 시작: RAGEvaluationDatasetTest (49개 항목)

🔄 아이템 1/49 처리 중...
   ✅ 완료 (종합 점수: 0.77)
   🔍 세부 정보: {'rouge': {'rouge1': 0.6799999999999999, 'rouge2': 0.45454545454545453, 'rougeL': 0.49000000000000005}, 'conciseness': {'reasoning': 'Step-by-step reasoning:\n\n1. The criterion to evaluate is conciseness: "Is the submission concise and to the point?"\n\n2. The submission provides the reason for the rejection of Elon Musk\'s $56 billion pay package: "부적절한 이사회 승인이라는 이유로 2024년 12월 델라웨어 법원에서 거부되었습니다." This is concise and clear.\n\n3. Regarding Elon Musk\'s early strategic role at Tesla, the submission mentions:\n   - In 2004, he led early funding.\n   - Focus on a strategy starting with premium sports cars.\n   - Became chairman and CEO in 2008.\n   - Pushed a business strategy starting with high-priced low-volume cars before expanding to mainstream vehicles.\n\n4. Comparing this with the reference, the reference is more concise:\n   - It m